# Lab 5

In our final lab, we will be building on topics we have covered previously and introducing some advanced new techniques. Rather than jumping from one AOI to another, we will be working with the same area to help demonstrate how different analyses can tell us different things about our imagery.

# Requirements

Everytime you open a new notebook (or your kernel disconnects) you will need to run these requirements before you are able to produce any maps.

The 'pip install' lines are commented out becuase most everyone should already have these libraries installed on your devices, but in case you do not, uncomment and run the first two lines.

In [ ]:
# ! pip install geemap
# ! pip install pygis

import geemap
import ee
from geemap import basemaps

ee.Authenticate()
ee.Initialize(project='YOUR_PROJECT_NAME') # Replace 'YOUR_PROJECT_NAME' with the name of your earth engine cloud project

# Graded Section

## Conceptual Questions (40%)

### Question 1 (10 Points)

Explain what temporal resolution means for satellite imagery. When mapping landscape disturbances, why is temporal accuracy so important?

### Question 2 (10 Points)

In the "Controlling for Seasonality" part of section 5.1, why did we compute the difference in difference images? How does this metric help us focus in on this natural disaster?

### Question 3 (10 Points)

How do you interpret a Tasseled Cap image like the ones we produced in section 5.2? What does it mean when a pixel is more red, green, or blue? Why are the clouds purple/pink?

### Question 4 (10 Points)

Explain what the GLCM function can tell us about our image. Would you use the B4 or B5 image to highlight roads? Why?

## Challenge Section (60%)

Map Helen Damage for another watershed in the TN/NC area or for a smaller division of the Nolichucky, following the flow of this lab. No need to do the 'Controlling for Seasonality' section, just do the 2024 images.

This will require you to produce both great images and great interpretations. You can't assume your viewer will see what you are seeing. In at least 4-5 sentences, explain the story your image layers are telling. (10 points)

For full credit (50 points!), your map will need AT LEAST each of the following layers:
- 2 true color images Landsat or Sentinel: one before your event and one after (10 points)
- 2 computed index images, one before and one after (10 points) -- can be NDVI, NDBI, NBR, etc.
- Difference image between index images (10 points)
- 2 Tasseled Cap images, one before and one after (20 points) NOTE: You will need to get Landsat images for this.

*Your interpretation should go here. Explain each image you produced and what it is showing.*

# 5.1 Disturbance Mapping

Earth's surface undergoes a lot of change at a lot of different scales. Over millions of years, biomes can transform. Over decades, cities can grow. Overnight, buildings and vegetation can be demolished. The more narrow your question is temporally, the more complicated image collection can get. Whatever your question and whatever the scale, the *when* is often more difficult than the *where*.

## Riparian Damage Post-Helene

At the end of September 2024, Hurricane Helene devastated much of Western North Carolina and parts of Eastern Tennessee. This storm dumped an extreme amount of rain on this area over just a couple of days, causing catastrophic flooding and damage along the many streams and rivers tasked with moving water out.

In [ ]:
# Create and Display Map
map = geemap.Map(basemap='HYBRID')
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Get the Nolichucky watershed from national layer and add to map
watersheds = ee.FeatureCollection("USGS/WBD/2017/HUC08").filter(ee.Filter.eq('huc8', '06010108'))

map.centerObject(watersheds, 9)
map.addLayer(watersheds, {}, "Watershed")

In [ ]:
# Since the event is at the end of September we try to get a pre-event image from
# within a month before, with as little clouds as possible
sentinelPre = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterDate('2024-09-15', '2024-09-30') \
  .filterBounds(watersheds) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .median()

# Clip image to the watershed layer
pre = sentinelPre.select('B.*').clip(watersheds)

# Compute NDVI from the clipped image: NIR (B8) and Red (B4)
preNDVI = pre.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Add true color image and NDVI image to map
map.addLayer(pre, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'Pre-event')
map.addLayer(preNDVI, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'Pre-event NDVI')

In [ ]:
# For our post-event image, we want to get as close to the day after the event
# as we can. You can play with the parameters, but this seems to get the best image.
sentinelPost = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterDate('2024-10-01', '2024-10-15') \
  .filterBounds(watersheds) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
  .median()

# Clip post image to watershed
post = sentinelPost.select('B.*').clip(watersheds)

# Compute NDVI
postNDVI = post.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Add layers to map
map.addLayer(post, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'Post-event')
map.addLayer(postNDVI, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'Post-event NDVI')

In [ ]:
# Get a difference image between post and pre NDVI
# Post - Pre = Effect of
dndvi = postNDVI.subtract(preNDVI)

map.addLayer(dndvi, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'green']}, 'DNDVI')

## Controlling for Seasonality

Although our NDVI Difference image for 2024 does a great job of showing us the change, we cannot be sure that change was due directly to Helene. To be a little more sure, we can compare this change to the change over the same time period the previous year. If our 2024 image is showing a greater degree of change around the riparian corridor, then we can start to assume Helene caused a lot of damage.

In [ ]:
map2 = geemap.Map(basemap='HYBRID')
map2.centerObject(watersheds, 9)
map2.addLayer(watersheds, {}, "Watershed")
map2

Map(center=[36.100447741014435, -82.57366242446554], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
sent_2023 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterDate('2023-09-01', '2023-09-30') \
  .filterBounds(watersheds) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .median()

sent_2023 = sent_2023.select('B.*').clip(watersheds)

sent_2023_ndvi = sent_2023.normalizedDifference(['B8', 'B4']).rename('NDVI')

map2.addLayer(sent_2023_ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, '2023 Pre NDVI')

In [ ]:
post_2023 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
  .filterDate('2023-10-01', '2023-10-15') \
  .filterBounds(watersheds) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
  .median()

post_2023 = post_2023.select('B.*').clip(watersheds)

post_2023_ndvi = post_2023.normalizedDifference(['B8', 'B4']).rename('NDVI')

map2.addLayer(post_2023_ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, '2023 Post NDVI')

In [ ]:
map3 = geemap.Map(basemap='HYBRID')
map3.centerObject(watersheds, 9)
map3.addLayer(watersheds, {}, "Watershed")
map3

dndvi_2023 = post_2023_ndvi.subtract(sent_2023_ndvi)

ddndvi = dndvi.subtract(dndvi_2023)

map3.addLayer(dndvi_2023, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'green']}, 'DNDVI 2023')
map3.addLayer(dndvi,{'min': -1, 'max': 1, 'palette': ['red', 'white', 'green']}, 'DNDVI 2024' )
map3.addLayer(ddndvi, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'green']}, 'DDNDVI')

map3

Map(center=[36.100447741014435, -82.57366242446554], controls=(WidgetControl(options=['position', 'transparent…

# 5.2 Image Enhancement for Change Detection

## Tasseled Cap (TC)

[Tasseled Cap (TC) transformation](https://supportsi.hexagon.com/s/article/Tasseled-Cap-Transformation-for-Landsat-8?language=en_US) is a widely used technique in remote sensing to compress spectral information from multispectral imagery into a few key components related to vegetation and soil characteristics. It was originally developed for Landsat imagery and involves a linear transformation of the original bands into new components often interpreted as Brightness, Greenness, and Wetness. These components can highlight different aspects of the landscape and are useful for tasks like vegetation monitoring, change detection, and land cover classification.

In [ ]:
tc = geemap.Map(basemap='HYBRID')
tc.centerObject(watersheds, 9)
tc.addLayer(watersheds, {}, "Watershed")
tc

Map(center=[36.100447741014435, -82.57366242446554], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Get Landsat 8 Collection 2, Level 2 data
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# Filter Landsat images for the pre-event period
# Since Landsat has a longer return period, we need to expand our "pre" range
landsat_pre = landsat8.filterDate('2024-08-01', '2024-09-30') \
    .filterBounds(watersheds) \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Filter Landsat images for the post-event period
landsat_post = landsat8.filterDate('2024-10-01', '2024-10-30') \
    .filterBounds(watersheds) \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# Apply scaling factors to the collections
landsat_pre = landsat_pre.map(apply_scale_factors)
landsat_post = landsat_post.map(apply_scale_factors)

# Mosaic the collections to get single images for each period
landsat_pre_mosaic = landsat_pre.median().clip(watersheds)
landsat_post_mosaic = landsat_post.median().clip(watersheds)

In [ ]:
# Tasseled Cap Analysis for Landsat 8
def add_tc_l8(img):
    # Landsat 8/9 OLI Tasseled Cap coefficients (Baig et al., 2014)
    band_names = img.bandNames().getInfo()
    B2,B3,B4,B5,B6,B7 = [img.select(x).toFloat() for x in ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']] # Cast bands to float

    # Tasseled Cap coefficients for Landsat 8
    b_coef = [0.3029, 0.2786, 0.4733, 0.5599, 0.5080, 0.1872] # Brightness
    g_coef = [-0.2941, -0.2430, -0.5424, 0.7276, 0.0713, -0.1608] # Greenness
    w_coef = [0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559] # Wetness

    bands = [B2, B3, B4, B5, B6, B7]

    # Calculate Tasseled Cap components using image.multiply() and image.add()
    TCB = ee.Image(0).toFloat().rename('TCB')
    TCG = ee.Image(0).toFloat().rename('TCG')
    TCW = ee.Image(0).toFloat().rename('TCW')

    for i in range(len(bands)):
        TCB = TCB.add(bands[i].multiply(b_coef[i]))
        TCG = TCG.add(bands[i].multiply(g_coef[i]))
        TCW = TCW.add(bands[i].multiply(w_coef[i]))


    return img.addBands([TCB, TCG, TCW])

# Calculate TCA for the mosaicked pre and post event Landsat images
landsat_pre_tca = add_tc_l8(landsat_pre_mosaic)
landsat_post_tca = add_tc_l8(landsat_post_mosaic)

# Define visualization parameters for Landsat TCA (adjust min/max as needed)
landsat_tca_vis = {
    'bands': ['TCB', 'TCG', 'TCW'],
    'min': -0.5,
    'max': 0.5
}

tc.addLayer(landsat_pre_tca, landsat_tca_vis, 'Landsat Pre-event')
tc.addLayer(landsat_post_tca, landsat_tca_vis, 'Landsat Post-event')

## Gray Level Co-occurence Matrix (GLCM)

[Gray Level Co-occurrence Matrix (GLCM)](https://developers.google.com/earth-engine/apidocs/ee-image-glcmtexture) is a statistical method used in image analysis to describe the texture of an image. It works by calculating how often different pairs of pixel gray-level values occur in a specified spatial relationship (e.g., distance and angle). From the GLCM, various texture metrics can be derived, such as contrast, correlation, energy (angular second moment), homogeneity (inverse difference moment), and more. These metrics provide insights into the spatial arrangement and variability of pixel values, which can be useful for tasks like land cover classification, object detection, and identifying areas with different surface textures. In other words, it's kind of like an X-ray of our image!

In [ ]:
# Create a new map for GLCM
glcm = geemap.Map(basemap='HYBRID')
glcm.centerObject(watersheds, 9)
glcm.addLayer(watersheds, {}, "Watershed")
glcm

Map(center=[36.100447741014435, -82.57366242446554], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Apply GLCM to the pre-event image
# Select a single band for GLCM, typically a near-infrared or red band
# Using B4 (Red) for GLCM as an example
# Cast the band to a supported integer type (e.g., int16)
glcm_image_1 = post.toInt32().glcmTexture(size=1)

# Define visualization parameters for GLCM
# GLCM produces multiple bands, but we will focus on the 'B*_contrast' for each
glcm_red = {
    'min': 0,
    'max': 50000,
    'bands': 'B4_contrast', # Red band contrast
    'palette': ['000000', 'FFFFFF'] # Example palette (black to white)
}

glcm_nir = {
    'min': 0,
    'max': 50000,
    'bands': 'B5_contrast', # Change to 'B5_contrast' to view NIR contrast
    'palette': ['000000', 'FFFFFF'] # Example palette (black to white)
}

# Add the GLCM layer to the new map
glcm.addLayer(glcm_image_1, glcm_red, 'GLCM Contrast (Red) Size 1')
glcm.addLayer(glcm_image_1, glcm_nir, 'GLCM Contrast (NIR) Size 1')

In [ ]:
glcm_image_2 = post.toInt32().glcmTexture(size=2)

red_2 = {
    'min': 0,
    'max': 50000,
    'bands': 'B4_contrast', # Red band contrast
    'palette': ['000000', 'FFFFFF'] # Example palette (black to white)
}

nir_2 = {
    'min': 0,
    'max': 50000,
    'bands': 'B5_contrast', # Change to 'B5_contrast' to view NIR contrast
    'palette': ['000000', 'FFFFFF'] # Example palette (black to white)
}

glcm.addLayer(glcm_image_2, red_2, 'GLCM Contrast (Red) Size 2')
glcm.addLayer(glcm_image_2, nir_2, 'GLCM Contrast (NIR) Size 2')